In [4]:
from gurobipy import Model, GRB, quicksum
import numpy as np

In [15]:

def check_colored_trails(M, k, c):
    n = M.shape[0]
    edges = [(i, j) for i in range(n) for j in range(n) if M[i, j] > 0]
    m = len(edges)
    
    model = Model("Colored Trails")
    
    # Decision variables: X[a, ij] for color a and edge ij
    X = model.addVars(c, m, vtype=GRB.BINARY, name="X")
    
    # Constraint 1: Edge weights in each color must be less than k
    for a in range(c):
        model.addConstr(quicksum(M[i, j] * X[a, e] for e, (i, j) in enumerate(edges)) <= k, f"Weight_Limit_{a}")
    
    p = {}
    for a in range(c):
        for u in range(n):
            incoming = quicksum(X[a, e] for e, (v, uu) in enumerate(edges) if uu == u)
            outgoing = quicksum(X[a, e] for e, (uu, v) in enumerate(edges) if uu == u)
            p[u, a] = incoming - outgoing
    
    # Constraint 2: Summation of p(u, a) over all u must be 0 for each a
    for a in range(c):
        model.addConstr(quicksum(p[u, a] for u in range(n)) == 0, f"Trails_{a}")
    
    # Constraint 3: Summation of p(u, a)^2 over all u must be 2 for each a
    for a in range(c):
        model.addConstr(quicksum(p[u, a] * p[u, a] for u in range(n)) == 2, f"Not_disjoint_{a}")
    
    # Constraint 4: Each edge must be assigned to exactly one color
    for e in range(m):
        model.addConstr(quicksum(X[a, e] for a in range(c)) == 1, f"Exactly_one_color{e}")
    
    # Optimize the model
    model.setParam(GRB.Param.OutputFlag, 0)
    model.optimize()
    
    # Check feasibility
    if model.status == GRB.OPTIMAL:
        solution = np.zeros((c, m))
        for a in range(c):
            for e in range(m):
                solution[a, e] = X[a, e].x
        return True, solution
    else:
        return False, None

In [21]:
# Trivial case : path
M = np.array([[0, 2, 0, 0],
              [0, 0, 3, 0],
              [0, 0, 0, 3],
              [0, 0, 0, 0]])
k = 8
c = 1

check_colored_trails(M, k, c)

(True, array([[1., 1., 1.]]))

In [22]:
def compute_c(M, k):
    n = M.shape[0]
    l = 1
    r = len([(i, j) for i in range(n) for j in range(n) if M[i, j] > 0])
    
    c = r
    while l <= r:
        mid = (l + r)//2
        feasible, _ = check_colored_trails(M, k, mid)
        if feasible : 
            c = min(c, mid)
            r = mid - 1
        else : 
            l = mid + 1
    return c

In [23]:
compute_c(M, k)

1